In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM mapping_categories WHERE categorie IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
PCLAT,6219,PERCHLORATE,Perchlorate,14797-73-0,perchlorate


In [3]:
%%sql
SELECT * FROM mapping_categories
WHERE libminparametre IN ('1,4 dioxane', 'perchlorate')

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,casparam,categorie
14DAN,1580,"1,4 DIOXANE","1,4 dioxane",123-91-1,hydrocarbures


In [4]:
%%sql
SELECT DISTINCT 
    cdparametresiseeaux, limitequal_float , limitequal
FROM int__resultats_udi_communes
WHERE cdparametresiseeaux IN ('14DAN','PCLAT')

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdparametresiseeaux,limitequal_float,limitequal
PCLAT,None,None
14DAN,None,None


In [5]:
%%sql --save int__resultats_sub_indus_udi_dernier

WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        cdparametresiseeaux IN ('14DAN', 'PCLAT')
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
)

SELECT
    last_pvl.cdreseau,
    'dernier_prel' AS periode,
    last_pvl.datetimeprel AS dernier_prel_datetime,
    last_pvl.valtraduite AS dernier_prel_valeur,
    1 AS nb_parametres,
    'sub_indus_' || last_pvl.cdparametresiseeaux AS categorie,
    CASE
        WHEN
            last_pvl.valtraduite = 0
            OR last_pvl.valtraduite IS NULL
            THEN 'non_quantifie'
        WHEN
            last_pvl.valtraduite >= 0.35
            AND last_pvl.cdparametresiseeaux = '14DAN'
            THEN 'sup_0_35'
        WHEN
            last_pvl.valtraduite < 0.35
            AND last_pvl.cdparametresiseeaux = '14DAN'
            THEN 'inf_0_35'
        WHEN
            last_pvl.valtraduite >= 15
            AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'sup_15'
        WHEN
            last_pvl.valtraduite >= 4 AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'sup_4'
        WHEN
            last_pvl.valtraduite < 4 AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'inf_4'
        WHEN
            last_pvl.valtraduite < 15 AND last_pvl.cdparametresiseeaux = 'PCLAT'
            THEN 'inf_15'
        ELSE 'error'
    END AS resultat
FROM
    last_pvl
WHERE
    last_pvl.row_number = 1



Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,periode,dernier_prel_datetime,dernier_prel_valeur,nb_parametres,categorie,resultat
002000045,dernier_prel,2024-10-22 09:58:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000136,dernier_prel,2024-10-23 12:13:00,1.700,1,sub_indus_PCLAT,inf_4
002000155,dernier_prel,2024-10-07 10:46:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000318,dernier_prel,2024-04-15 09:39:00,6.100,1,sub_indus_PCLAT,sup_4
002000463,dernier_prel,2024-07-02 11:24:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000583,dernier_prel,2024-04-22 09:02:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000716,dernier_prel,2024-05-29 09:16:00,2.900,1,sub_indus_PCLAT,inf_4
002000824,dernier_prel,2024-07-01 12:25:00,2.800,1,sub_indus_PCLAT,inf_4
002000925,dernier_prel,2024-11-20 10:00:00,6.100,1,sub_indus_PCLAT,sup_4
002000963,dernier_prel,2024-07-17 10:03:00,0.000,1,sub_indus_PCLAT,non_quantifie


In [6]:
%%sql --with int__resultats_sub_indus_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_sub_indus_udi_dernier
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,periode,dernier_prel_datetime,dernier_prel_valeur,nb_parametres,categorie,resultat


In [7]:
%%sql --with int__resultats_sub_indus_udi_dernier
SELECT 
    resultat, COUNT(*) 
FROM 
    int__resultats_sub_indus_udi_dernier
GROUP BY 
    resultat

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,count_star()
sup_15,29
inf_4,429
non_quantifie,194
sup_0_35,1
sup_4,182


In [8]:
%%sql --with int__resultats_sub_indus_udi_dernier
SELECT 
    resultat, MIN( cdreseau) min_cdreseau, MAX(cdreseau) AS max_cdreseau
FROM 
    int__resultats_sub_indus_udi_dernier
GROUP BY 
    resultat

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min_cdreseau,max_cdreseau
inf_4,002000048,080003913
non_quantifie,002000045,080001512
sup_0_35,052000760,052000760
sup_15,002000114,080003918
sup_4,002000060,088002246


In [9]:
%%sql --with int__resultats_sub_indus_udi_dernier 
 SELECT 
   *
 FROM
   int__resultats_sub_indus_udi_dernier
WHERE
    cdreseau IN( '002000048','080003913','002000045','080001512','052000760','002000114','080003918','002000060','088002246')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,periode,dernier_prel_datetime,dernier_prel_valeur,nb_parametres,categorie,resultat
002000045,dernier_prel,2024-10-22 09:58:00,0.000,1,sub_indus_PCLAT,non_quantifie
002000048,dernier_prel,2024-04-22 10:10:00,1.900,1,sub_indus_PCLAT,inf_4
002000060,dernier_prel,2024-10-18 08:31:00,8.300,1,sub_indus_PCLAT,sup_4
002000114,dernier_prel,2024-10-10 11:03:00,45.000,1,sub_indus_PCLAT,sup_15
052000760,dernier_prel,2024-12-13 14:52:00,0.840,1,sub_indus_14DAN,sup_0_35
080001512,dernier_prel,2024-05-02 14:30:00,0.000,1,sub_indus_PCLAT,non_quantifie
080003913,dernier_prel,2024-05-31 11:23:00,2.200,1,sub_indus_PCLAT,inf_4
080003918,dernier_prel,2024-10-23 13:09:00,29.000,1,sub_indus_PCLAT,sup_15
088002246,dernier_prel,2024-10-22 10:33:00,9.200,1,sub_indus_PCLAT,sup_4
